<center><img src="https://datascientest.fr/train/assets/logo_datascientest.png", style="height:150px"/></center>

<hr style="border-width:2px;border-color:#75DFC1">

<center><img src="https://assets-datascientest.s3.eu-west-1.amazonaws.com/logo+SatisPy+Project.png", style="height:150px"/></center>


<center><h1> Modèles - Analyse & Tableaux d'erreurs </h1></center>

# Objectifs :

L'objectif est de pouvoir lancer plusieurs modélisations sur notre data_set original, ceux filtrés ou sur d'autres variables préparées
- Utiliser des classifier de bases du Machine Learning
- Utiliser des modèles pré-entrainés pour y ajouter nos modèles prédictifs
- Utiliser nos bases acquises en Deep Learning pour modéliser des réseaux de neurones
- Comparer les scores
- Comparer les erreurs dans un tableau spécifiques
- lancer des prédictions rapides à la volée

In [1]:
# chargement des packages
import sys
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline 
from datetime import datetime
from time import time
import time 
import pickle
import nltk
from nltk.tokenize import PunktSentenceTokenizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter
import langid                      # langid non plus
from wordcloud import WordCloud
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import GradientBoostingClassifier
from nltk.corpus import stopwords
from sklearn.naive_bayes import BernoulliNB,CategoricalNB
from sklearn.svm import SVC
from sklearn.metrics import *
from sklearn.metrics import classification_report,confusion_matrix
import tensorflow
from tensorflow.keras.layers import Input, Dense #Pour instancier une couche Dense et une d'Input
from tensorflow.keras.models import Model
import spacy

# Versions
print('Version des librairies utilisées :')
print('Python                : ' + sys.version)
print('NumPy                 : ' + np.version.full_version)
print('Pandas                : ' + pd.__version__)
print('Matplotlib.pyplot     : ' + matplotlib.__version__)
print('Seaborn               : ' + sns.__version__)
print('NaturaLanguageTooKit  : ' + nltk.__version__)
print('sklearn               : ' + sklearn.__version__)
print('Tensorflow            : ' + tensorflow.__version__)
print('Spacy                 : ' + spacy.__version__)


# préparation de l'affichage des dataframes => même fonction que le notebook data_explo à séparer dans un outils.py à importer

dico_pd_option = {
    'display.max_rows': 100,    # nbre max de lignes 
    'display.max_column': 50,  # nbre max de colonnes
    'display.width': 50,       # largeur lignes 
    'display.precision': 2,     # precision des valeurs
    'display.max_colwidth': 100  # largeurs colonnes
}
for cle, val in dico_pd_option.items():
    pd.set_option(cle, val)
    
pd.options.display.float_format = "{:,.3f}".format  # affichage pd float à 3 décimales

Version des librairies utilisées :
Python                : 3.9.12 (main, Apr  4 2022, 05:22:27) [MSC v.1916 64 bit (AMD64)]
NumPy                 : 1.21.5
Pandas                : 1.4.2
Matplotlib.pyplot     : 3.5.1
Seaborn               : 0.11.2
NaturaLanguageTooKit  : 3.7
sklearn               : 1.0.2
Tensorflow            : 2.9.1
Spacy                 : 3.4.1


# 4- Modélisation des données

Il y a 9 modèles d'apprentissage appliqué dans le projet. Les données d'entrées sont différents. 
Notre rapport de projet en a sélectionné seulement 7 pour éviter les redondances inutiles

Pour faire la modélisation, les étapes principales : 
* 1- Préparation des données
* 2- Construction du modèle
* 3- Modélisation des données
* 4- Évaluation du modèle

<hr style="border-width:2px;border-color:#75DFC1">
<center><h1>1ère et 2ème modélisation </h1></center>
<hr style="border-width:2px;border-color:#75DFC1">

# Modèle 1 - Logistic Regression



In [2]:
# importer les données nettoyées
df = pd.read_csv('reviews_trust_fr_VF.csv',index_col = 0)
df.head()

,Commentaire,star,source,company,index_org,star_org,date
0,"bonjour , ca doit faire 5 ans environ que je suis membre showrooprive . je n ’ ai jamais eu de s...",1,TrustPilot,ShowRoom,0,1,2021-06-20 00:00:00+00:00
1,vente lacoste article manquant photo prise sur 6 articles la moitié livrée sans explication manq...,1,TrustPilot,ShowRoom,1,1,2021-06-20 00:00:00+00:00
2,"vente lacoste honteuse , article erroné , article manquant , pas de bon de livraison , retard et...",1,TrustPilot,ShowRoom,2,1,2021-06-19 00:00:00+00:00
3,"j'ai commandé des mules de la marque moosefield , très déçue du produit , une étiquette collée s...",2,TrustPilot,ShowRoom,3,2,2021-06-19 00:00:00+00:00
4,commande téléphone etat a+ . livraison d un vieux téléphone pourri sans batterie rayé partout et...,1,TrustPilot,ShowRoom,4,1,2021-06-19 00:00:00+00:00


##  1- Préparation des données

Les étapes principales : 

* a- raffiner les commentaires 
* b- traiter les dates
* c- traiter les star, company, source

### a- raffiner les commentaires

In [3]:
def find_exclamation(Commentaire):
    r = re.compile(r"\!")
    exclamation = r.findall(Commentaire)
    return len(exclamation)

def find_interogation(Commentaire):
    r = re.compile(r"\?")
    interogation = r.findall(Commentaire)
    return len(interogation)

def find_etc(Commentaire):
    r = re.compile(r"\.{2,}")
    etc = r.findall(Commentaire)
    return len(etc)

df['exclamation'] = df.Commentaire.apply(lambda x: find_exclamation(x))
df['interogation'] = df.Commentaire.apply(lambda x: find_interogation(x))
df['etc'] = df.Commentaire.apply(lambda x: find_etc(x))
df['nb_caracter'] = df.Commentaire.apply(len)

### b- traiter les dates

In [4]:
df['date'] = pd.to_datetime(df['date'],utc = True)
df['year'] = pd.to_datetime(df['date']).dt.year
df['month'] = pd.to_datetime(df['date']).dt.month
df['weekday'] = pd.to_datetime(df['date']).dt.weekday
df.drop('date', axis = 1, inplace = True)


### c- traiter les star, company, source

* positive = 1 corresepond star 3, 4 et 5
* négative = 0 correspond start 1 et 2

In [5]:
new_star = {1:0,2:0,3:1,4:1,5:1}
df.star.replace(new_star,inplace = True)

In [6]:
# observation sur les entrées
# objectifs : vérifier si les entrées ont un sens 
# display(df[['star', 'exclamation', 'chain_exclamation', 'interogation', 'nb_caracter']].groupby(['star']).sum())
display(df[['star', 'exclamation', 'interogation', 'nb_caracter']].groupby(['star']).sum())


,exclamation,interogation,nb_caracter
star,,,
0,12901,1232,2947360
1,2549,218,990510


In [7]:
df= df.join(pd.get_dummies(df['source'],prefix = 'source'))
df= df.join(pd.get_dummies(df['company'],prefix = 'company'))
df.drop(['source','company'], axis= 1, inplace = True)

In [8]:
df.head()
df.to_csv('reviews_trust_fr_LR.csv')
# sauvegarder les données pour alimenter le 3ème modèle BernoulliNB()

Jusqu'à ici, la quantification des informations (*data, commentaire, source, star*) est finie. 

### 2- Construction du modèle
### 3- Modélisation des données
- entrées (data/X): 'exclamation', 'chain_exclamation', 'interogation', 'nb_caracter','source','company','year','month','weekday'
   
- sortie (target/Y): 'star'

In [9]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(['Commentaire','star','index_org','star_org'], axis=1), df.star, test_size=0.2, random_state=20)


In [10]:
logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

### 4- Evaluation du modèle

In [11]:
# les score : 
print('Score :',logreg.score(X_test, y_test))

Score : 0.8877906976744186


In [12]:
# cross table
y_pred = logreg.predict(X_test)
pd.crosstab(y_test, y_pred, rownames = ['real'], colnames =['predict'])

predict,0,1
real,,
0,895,277
1,109,2159


In [13]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.89      0.76      0.82      1172
           1       0.89      0.95      0.92      2268

    accuracy                           0.89      3440
   macro avg       0.89      0.86      0.87      3440
weighted avg       0.89      0.89      0.89      3440



### Création et sauvegarde du tableau des erreurs de prédiction pour analyse ultérieure

In [14]:
# on récupère les données complètes du test_set avec indexs et star d'origine
# création d'un dataframe vide
df_analyse = pd.DataFrame()  
# recupérer tou le y_test en renommant la colonne star par star_1 pour modele 1
df_analyse['star_1'] = y_test     
# ajouter une colonne d'index ordonnée de 0 à taille du df et déplacer les index d'erreurs dans une colonne du df
df_analyse.reset_index(inplace=True, drop=False)
# renommer la colonne d'index d'erreurs par index_1 pour modele 1
df_analyse.rename(columns={'index':'index_1'}, inplace=True)
# ajouter au df la colonnede prédiction réalisée sur y_test
df_analyse['pred_1'] = pd.Series(y_pred)                                        


# filtrage des erreurs de prediction par boolean indexing
 # création d'une liste boolean vérifiant sur les prédiction sont bonnes ou mauvaises       
bool_pred = [True if (df_analyse.iloc[i][1]==df_analyse.iloc[i][2]) else False for i in range(df_analyse.shape[0])]
# ajout de la liste du résultat booleen dans le tableau d'analyse des erreurs
df_analyse['bool_1'] = pd.DataFrame(bool_pred)
# création d'un dataframe des seules erreurs du modele
df_verification = df_analyse.loc[df_analyse['bool_1']==False]   # Parfait 386 erreurs aussi ici de pred facilement trouvable avec vrais index
# effacement des index non originaux copiés et sans intéret
df_verification.reset_index(inplace=True, drop=True)  


# Rajouter la colonne de commentaires originaux + colones commentaires filtrés finaux à df_ verification
list_com_vo = []
for i in range(df_verification.shape[0]):
    find_index = df_verification['index_1'][i]
    list_com_vo.append(df['Commentaire'][find_index])
    
# ajout commentaires originaux    
df_verification['comment_vo'] = pd.Series(list_com_vo)       # Parfait , les commentaires sont aux bons index !

C:\Users\UserPC\AppData\Local\Temp\ipykernel_11720\964225027.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_verification['comment_vo'] = pd.Series(list_com_vo)       # Parfait , les commentaires sont aux bons index !


In [15]:
# affichage du tableau comparatif des erreurs réalisé
print("df_verification.shape = ", df_verification.shape)
df_verification.head(10)

df_verification.shape =  (386, 5)


,index_1,star_1,pred_1,bool_1,comment_vo
0,14776,1,0,False,"problème avec un réveil qui ne fonctionnait plus.une étiquette retour m ' a été envoyée , colis ..."
1,15017,1,0,False,"excellent site de e commerce , imbattable , tout simplement ! !"
2,10844,0,1,False,"cette commande a cumulé les problèmes . j'ai été livrée très tard , bien plus tard que le délai ..."
3,7751,0,1,False,je n'ai reçu que la moitié de ma commande pourtant validée par vos soins . dès lors où vos engag...
4,15263,1,0,False,je n'ai jamais été déçu . .. j'achète en toute confiance
5,13505,0,1,False,pas du tout comme sur la photo
6,15054,1,0,False,je commande très souvent et pour l'instant je n'ai pas eu bcp de problèmes : )
7,9117,0,1,False,"bijoux pas du tout comme sur la photo , cheap , les prix annoncés avec un rabais de -75 % sont f..."
8,413,1,0,False,je suis déçue concernant ma commande showroom . après avoir commandé une paire de bottes blackfi...
9,13150,0,1,False,livraison arrivée défectueuse pour un colis : emballage de paquets de denrées alimentaires eclat...


### Nous retrouvons bien les 386 erreurs identifiées dans la matrice de confusion du modele 1

# Modèle 2- Gradient Boosting Classifier
* Modèle 2.1- sans utiliser le regex et stop words pour faire tokenizer
* Modèle 2.2- en utilisant le regex et stop words pour faire tokenizer


In [16]:
# importer les données nettoyées 
df = pd.read_csv('reviews_trust_fr_VF.csv',index_col = 0)
# entrée : Commentaire
# cible : star (0, 1)
new_star = {1:0,2:0,3:1,4:1,5:1}
df.star.replace(new_star,inplace = True)
features = df.Commentaire
target = df.star.astype('int')
X_train_org,X_test_org,y_train, y_test = train_test_split(features, target, test_size = 0.2,random_state=20)

**NOTE**: Ici deux modèles GBC avec ou sans filtrage de commentaires sont utilisés. Pour comparer les résultats, les données d'entrées sont les même.

### Modèle 2.1 
Appliquer GBC a *commentaire* sans utiliser le regex et stop words pour faire tokenizer
* 1- Préparation des données
* 2- Construction du modèle
* 3- Modélisation des données
* 4- Evaluation du modèle

In [17]:
%%time

vectorizer = CountVectorizer()

X_train_GBC_1 = X_train_org # utiliser les commentaires origines pour modéliser
X_test_GBC_1 = X_test_org   # utiliser les commentaires origines pour modéliser

# Mettre à jour la valeur de X_train et X_test
X_train_GBC_1 = vectorizer.fit_transform(X_train_GBC_1).todense()
X_test_GBC_1 = vectorizer.transform(X_test_GBC_1).todense()

GBC_1 = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0).fit(X_train_GBC_1, y_train)

# sauvegarder le modèle pré-entrainé
pickle.dump(GBC_1,open('GBC_1.pickle','wb')) 


C:\Users\UserPC\anaconda3\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


CPU times: total: 17min 14s
Wall time: 22min 7s


In [18]:
# sauvegarder le Vectoriser 
pickle.dump(vectorizer, open('vectoriser_GBC_1','wb'))


In [19]:
%%time 
# Calculer les prédictions 
y_pred_GBC_1 = GBC_1.predict(X_test_GBC_1)

#  Importer la classe classification_report
from sklearn.metrics import classification_report

# Calcul et affichage de classification_report
print(classification_report(y_test, y_pred))

# Calcul et affichage de la matrice de confusion
confusion_matrix = pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite'])
confusion_matrix

C:\Users\UserPC\anaconda3\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


              precision    recall  f1-score   support

           0       0.89      0.76      0.82      1172
           1       0.89      0.95      0.92      2268

    accuracy                           0.89      3440
   macro avg       0.89      0.86      0.87      3440
weighted avg       0.89      0.89      0.89      3440

CPU times: total: 781 ms
Wall time: 1.44 s


Classe prédite,0,1
Classe réelle,,
0,895,277
1,109,2159


### Création et sauvegarde du tableau des erreurs de prédiction pour analyse ultérieure

In [21]:
# on récupère les données complètes du test_set avec indexs et star d'origine
# création d'un dataframe vide
df_analyse = pd.DataFrame()  
# recupérer tou le y_test en renommant la colonne star par star_2 pour modele 2
df_analyse['star_2'] = y_test     
# ajouter une colonne d'index ordonnée de 0 à taille du df et déplacer les index d'erreurs dans une colonne du df
df_analyse.reset_index(inplace=True, drop=False)
# renommer la colonne d'index d'erreurs par index_2 pour modele 2
df_analyse.rename(columns={'index':'index_2'}, inplace=True)
# ajouter au df la colonnede prédiction réalisée sur y_test
df_analyse['pred_2'] = pd.Series(y_pred)                                        


# filtrage des erreurs de prediction par boolean indexing
 # création d'une liste boolean vérifiant sur les prédiction sont bonnes ou mauvaises       
bool_pred = [True if (df_analyse.iloc[i][1]==df_analyse.iloc[i][2]) else False for i in range(df_analyse.shape[0])]
# ajout de la liste du résultat booleen dans le tableau d'analyse des erreurs
df_analyse['bool_2'] = pd.DataFrame(bool_pred)
# création d'un dataframe des seules erreurs du modele
df_verification = df_analyse.loc[df_analyse['bool_2']==False]   # Parfait 386 erreurs aussi ici de pred facilement trouvable avec vrais index
# effacement des index non originaux copiés et sans intéret
df_verification.reset_index(inplace=True, drop=True)  


# Rajouter la colonne de commentaires originaux + colones commentaires filtrés finaux à df_ verification
list_com_vo = []
for i in range(df_verification.shape[0]):
    find_index = df_verification['index_2'][i]
    list_com_vo.append(df['Commentaire'][find_index])
    
# ajout commentaires originaux    
df_verification['comment_vo'] = pd.Series(list_com_vo) 

# affichage du tableau comparatif des erreurs réalisé
print("df_verification.shape = ", df_verification.shape)
df_verification.head(10)

df_verification.shape =  (386, 5)


C:\Users\UserPC\AppData\Local\Temp\ipykernel_11720\973380191.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_verification['comment_vo'] = pd.Series(list_com_vo)


,index_2,star_2,pred_2,bool_2,comment_vo
0,14776,1,0,False,"problème avec un réveil qui ne fonctionnait plus.une étiquette retour m ' a été envoyée , colis ..."
1,15017,1,0,False,"excellent site de e commerce , imbattable , tout simplement ! !"
2,10844,0,1,False,"cette commande a cumulé les problèmes . j'ai été livrée très tard , bien plus tard que le délai ..."
3,7751,0,1,False,je n'ai reçu que la moitié de ma commande pourtant validée par vos soins . dès lors où vos engag...
4,15263,1,0,False,je n'ai jamais été déçu . .. j'achète en toute confiance
5,13505,0,1,False,pas du tout comme sur la photo
6,15054,1,0,False,je commande très souvent et pour l'instant je n'ai pas eu bcp de problèmes : )
7,9117,0,1,False,"bijoux pas du tout comme sur la photo , cheap , les prix annoncés avec un rabais de -75 % sont f..."
8,413,1,0,False,je suis déçue concernant ma commande showroom . après avoir commandé une paire de bottes blackfi...
9,13150,0,1,False,livraison arrivée défectueuse pour un colis : emballage de paquets de denrées alimentaires eclat...


In [22]:
# sauvegarde des erreurs à analyser sur le model2
df_verification.to_csv('sav_verif_erreurs_model2.csv')

### Prédiction à la demande sur GBD 1 (en direct) 

In [27]:
# prediction à la demande
# charger le modèle pré-entrainé sauvegardé
GBC_1 = pickle.load(open('GBC_1.pickle','rb'))
# text = "la livraison est lente, le produit est abîmé"
text = "mauvais produit"
text= pd.Series(text)
text_vec = vectorizer.transform(text).todense()
print(text_vec)
pred = GBC_1.predict(text_vec)
str(pred)

[[0 0 0 ... 0 0 0]]


C:\Users\UserPC\anaconda3\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


'[1]'

- mauvaise prédiction => erreur dans le code ? ou normal car peu de mots => à creuser avec les tableaux d'erreurs et faire d'autres tests

### Modèle 2.2 
Appliquer GBC a commentaire en utilisant le reg et stop words pour faire tokenizer
* 1- Préparation des données
* 2- Construction du modèle
* 3- Modélisation des données
* 4- Evaluation du modèle

In [24]:
%%time
X_train_GBC_2 = X_train_org
X_test_GBC_2 = X_test_org

# traitement des mots vides #
# chargement fichier excel de stop words french en dataframe
df_stop_word_xls = pd.read_excel('liste_no-stop-words_tokens_unique.xlsx', header=None)

# création de stop words set

# # liste selon le retour d'expérience projet
update_list_fr = list(df_stop_word_xls[0])

# initialisation de la variable des mots vides
stop_words = set()
stop_words.update(update_list_fr)

# Initialiser un objet vectorizer, garde aussi un mot avec plus 2 lettres
vectorizer = CountVectorizer(stop_words = stop_words,token_pattern = r"[a-zéèêàâîïàùçôëûæœ]{2,}" )

# Mettre à jour la valeur de X_train et X_test
X_train_GBC_2 = vectorizer.fit_transform(X_train_GBC_2).todense()
X_test_GBC_2 = vectorizer.transform(X_test_GBC_2).todense()

# print(vectorizer.vocabulary_)
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import GradientBoostingClassifier
GBC_2 = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0).fit(X_train_GBC_2, y_train)

# sauvegarder le modèle pré-entrainé
pickle.dump(GBC_2,open('GBC_2.pickle','wb')) 
# sauvegarder le Vectoriser
pickle.dump(vectorizer, open('vectoriser_GBC_2','wb'))


C:\LQ\softwares\Anaconda\lib\site-packages\sklearn\utils\validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


Wall time: 4min 7s


In [25]:
%%time 
# Calculer les prédictions 
y_pred_GBC_2 = GBC_2.predict(X_test_GBC_2)
#  Importer la classe classification_report
from sklearn.metrics import classification_report

# Calcul et affichage de classification_report
print(classification_report(y_test, y_pred) )

# Calcul et affichage de la matrice de confusion
confusion_matrix = pd.crosstab(y_test, y_pred_GBC_2, rownames=['Classe réelle'], colnames=['Classe prédite'])
confusion_matrix

C:\LQ\softwares\Anaconda\lib\site-packages\sklearn\utils\validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


              precision    recall  f1-score   support

           0       0.89      0.76      0.82      1172
           1       0.89      0.95      0.92      2268

    accuracy                           0.89      3440
   macro avg       0.89      0.86      0.87      3440
weighted avg       0.89      0.89      0.89      3440

Wall time: 233 ms


Classe prédite,0,1
Classe réelle,,
0,901,271
1,135,2133


# Modèle 3 - Naive Bayes classifier for multivariate Bernoulli models
 ### -- BernoulliNB()

In [14]:
# importer les données nettoyées 
df = pd.read_csv('reviews_trust_fr_LR.csv',index_col = 0)
# utiliser les données quantitavie du Logistic regression

In [23]:
df.head()

,Commentaire,star,source,company,index_org,star_org,date
0,"bonjour , ca doit faire 5 ans environ que je suis membre showrooprive . je n ’ ai jamais eu de s...",0,TrustPilot,ShowRoom,0,1,2021-06-20 00:00:00+00:00
1,vente lacoste article manquant photo prise sur 6 articles la moitié livrée sans explication manq...,0,TrustPilot,ShowRoom,1,1,2021-06-20 00:00:00+00:00
2,"vente lacoste honteuse , article erroné , article manquant , pas de bon de livraison , retard et...",0,TrustPilot,ShowRoom,2,1,2021-06-19 00:00:00+00:00
3,"j'ai commandé des mules de la marque moosefield , très déçue du produit , une étiquette collée s...",0,TrustPilot,ShowRoom,3,2,2021-06-19 00:00:00+00:00
4,commande téléphone etat a+ . livraison d un vieux téléphone pourri sans batterie rayé partout et...,0,TrustPilot,ShowRoom,4,1,2021-06-19 00:00:00+00:00


In [15]:
# bag of words 
# from sklearn.feature_extraction.text import CountVectorizer
# from nltk.corpus import stopwords
# from sklearn.naive_bayes import MultinomialNB,GaussianNB,BernoulliNB,ComplementNB,CategoricalNB

# Split data in train and validate dataset.
df_train, df_test, y_train, y_test = train_test_split(df.drop(['star','index_org','star_org'], axis=1), 
                                                      df.star, test_size=0.2, random_state=20)


# traitement des mots vides #
# chargement fichier excel de stop words french en dataframe
df_stop_word_xls = pd.read_excel('liste_no-stop-words_tokens_unique.xlsx', header=None)

# création de stop words set

# # liste selon le retour d'expérience projet
update_list_fr = list(df_stop_word_xls[0])

# initialisation de la variable des mots vides
stop_words = set()
stop_words.update(update_list_fr)

# Bag of words, garde le mot avec au moins deux lettres
vectorizer = CountVectorizer(stop_words=stop_words, token_pattern = r"[a-zéèêàâîïàùçôëûæœ]{2,}",max_features=5000)

X_train_text = vectorizer.fit_transform(df_train.Commentaire)
X_test_text = vectorizer.transform(df_test.Commentaire)

from scipy.sparse import hstack
# hstack(): concater le text vectoriser (x_train_text) avec les autres informations quantitative 
# (comme data, year, nombre d'exclamation ) 
X_train = hstack((X_train_text, df_train.drop('Commentaire', axis=1).values))
X_test = hstack((X_test_text, df_test.drop('Commentaire', axis=1).values))

from sklearn.naive_bayes import BernoulliNB
ber = BernoulliNB()
ber.fit(X_train, y_train)

# sauvegarder le modèle entrainé
pickle.dump(ber,open('ber.pickle','wb')) 
# sauvegarder le Vectoriser
pickle.dump(vectorizer, open('vectoriser_ber','wb'))

# @fred: je ne pense pas que nous allons utiliser ce modèle pour predire une commentaire, 
# car il faut reproduire les données statistiques comme data, day etc
# à supprimer après ta lecture merci :) 

In [16]:
print('Score :',ber.score(X_test, y_test))
# Calculer les prédictions 
y_pred_ber = ber.predict(X_test)
# Calcul et affichage de la matrice de confusion
confusion_matrix = pd.crosstab(y_test, y_pred_ber, rownames=['Classe réelle'], colnames=['Classe prédite'])
confusion_matrix

Score : 0.8825581395348837


Classe prédite,0,1
Classe réelle,,
0,908,264
1,140,2128


In [17]:
# Calcul et affichage de classification_report
print(classification_report(y_test, y_pred) )

              precision    recall  f1-score   support

           0       0.89      0.76      0.82      1172
           1       0.89      0.95      0.92      2268

    accuracy                           0.89      3440
   macro avg       0.89      0.86      0.87      3440
weighted avg       0.89      0.89      0.89      3440



### Création et sauvegarde du tableau des erreurs de prédiction pour analyse ultérieure

In [24]:
# on récupère les données complètes du test_set avec indexs et star d'origine
# création d'un dataframe vide
df_analyse = pd.DataFrame()  
# recupérer tou le y_test en renommant la colonne star par star_3 pour modele 3
df_analyse['star_3'] = y_test     
# ajouter une colonne d'index ordonnée de 0 à taille du df et déplacer les index d'erreurs dans une colonne du df
df_analyse.reset_index(inplace=True, drop=False)
# renommer la colonne d'index d'erreurs par index_2 pour modele 2
df_analyse.rename(columns={'index':'index_3'}, inplace=True)
# ajouter au df la colonnede prédiction réalisée sur y_test
df_analyse['pred_3'] = pd.Series(y_pred)                                        


# filtrage des erreurs de prediction par boolean indexing
 # création d'une liste boolean vérifiant sur les prédiction sont bonnes ou mauvaises       
bool_pred = [True if (df_analyse.iloc[i][1]==df_analyse.iloc[i][2]) else False for i in range(df_analyse.shape[0])]
# ajout de la liste du résultat booleen dans le tableau d'analyse des erreurs
df_analyse['bool_3'] = pd.DataFrame(bool_pred)
# création d'un dataframe des seules erreurs du modele
df_verification = df_analyse.loc[df_analyse['bool_3']==False]   # Parfait 386 erreurs aussi ici de pred facilement trouvable avec vrais index
# effacement des index non originaux copiés et sans intéret
df_verification.reset_index(inplace=True, drop=True)  


# Rajouter la colonne de commentaires originaux + colones commentaires filtrés finaux à df_ verification
list_com_vo = []
for i in range(df_verification.shape[0]):
    find_index = df_verification['index_3'][i]
    list_com_vo.append(df['Commentaire'][find_index])
    
# ajout commentaires originaux    
df_verification['comment_vo'] = pd.Series(list_com_vo)

C:\Users\UserPC\AppData\Local\Temp\ipykernel_11720\2969940981.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_verification['comment_vo'] = pd.Series(list_com_vo)


In [25]:
# affichage du tableau comparatif des erreurs réalisé
print("df_verification.shape = ", df_verification.shape)
df_verification.head(10)

df_verification.shape =  (386, 5)


,index_3,star_3,pred_3,bool_3,comment_vo
0,14776,1,0,False,"problème avec un réveil qui ne fonctionnait plus.une étiquette retour m ' a été envoyée , colis ..."
1,15017,1,0,False,"excellent site de e commerce , imbattable , tout simplement ! !"
2,10844,0,1,False,"cette commande a cumulé les problèmes . j'ai été livrée très tard , bien plus tard que le délai ..."
3,7751,0,1,False,je n'ai reçu que la moitié de ma commande pourtant validée par vos soins . dès lors où vos engag...
4,15263,1,0,False,je n'ai jamais été déçu . .. j'achète en toute confiance
5,13505,0,1,False,pas du tout comme sur la photo
6,15054,1,0,False,je commande très souvent et pour l'instant je n'ai pas eu bcp de problèmes : )
7,9117,0,1,False,"bijoux pas du tout comme sur la photo , cheap , les prix annoncés avec un rabais de -75 % sont f..."
8,413,1,0,False,je suis déçue concernant ma commande showroom . après avoir commandé une paire de bottes blackfi...
9,13150,0,1,False,livraison arrivée défectueuse pour un colis : emballage de paquets de denrées alimentaires eclat...


In [26]:
# sauvegarde des erreurs à analyser sur le model3
df_verification.to_csv('sav_verif_erreurs_model3.csv')


<hr style="border-width:2px;border-color:#75DFC1">
<center><h1>3ème Modélisation avec pré-entrainement du corpus documents</h1></center>
<hr style="border-width:2px;border-color:#75DFC1">

#### 1- passage du corpus sur un modèle pré-entrainé en langue Française provenant de wikipedia de 2.500.733 de mots vectorisés
#### 2- modélisation et prédiction par SVM() à noyau RBF
#### 3- modélisation et prédiction par GBC 
#### 4- modélisation et prédiction par BernoulliNB()
#### 5- modélisation et prédiction par les réseaux neurones (CNN, RNN..)
#### 6- tableau de comprehension des erreurs de prédiction avec leur corpus original et filtré
#### 7- traitement et prédiction d'un commentaire 'à la volée'
#### 8- analyse finale et pistes d'améliorations

In [29]:
# importer les données nettoyées
data = pd.read_csv('review_trust_fr_lemmantiser_word+2_VF.csv', sep=',')  
# à corriger +2
# test reviews_trust_fr_VF.csv 
data.drop(data.iloc[:,:1], axis=1, inplace=True)
data.sample(1)

,Commentaire,star,source,company,index_org,star_org,date,tokens,lemmatiser,words+2,no_stop_words
14490,"bonjour , j'ai fait une commande le 25 avril 2020 de 2 pyjamas eminence pour l'anniversaire de m...",0,TrustPilot,VeePee,17677,1,2020-06-04 00:00:00+00:00,"['bonjour', 'j', 'ai', 'fait', 'une', 'commande', 'le', 'avril', 'de', 'pyjamas', 'eminence', 'p...","['bonjour', 'j', 'avoir', 'faire', 'un', 'commande', 'le', 'avril', 'de', 'pyjama', 'eminence', ...","['bonjour', 'avoir', 'faire', 'commande', 'avril', 'pyjama', 'eminence', 'pour', 'anniversaire',...","['bonjour', 'avoir', 'faire', 'commande', 'avril', 'pyjama', 'pour', 'anniversaire', 'mon', 'com..."


## 2- passage du corpus sur un modèle pré-entrainé en langue Française provenant de wikipedia de 2.500.733 de mots vectorisés
### Chargement du modèle pré-entrainé de wikipedia2vec
- TUTO d'aide pour charger une réprésentation pré-entraînée dans une langue souhaitée de wikipedia2vec

   - 1 aller sur le lien @  https://wikipedia2vec.github.io/wikipedia2vec/pretrained/#french
   - 2 copier le lien d'adresse d'un des 4 fichiers souhaité en langue Française de la ligne
     frwiki_20180420 (window=5, iteration=10, negative=15): 100d (bin) 100d (txt) 300d (bin) 300d (txt)
   - 3 placer le lien sur notre IDE spyder ou jupyter
   - 4 avec la souris sur ce chemin, faire CRL + clic => le fichier se télécharge = 1h pour le + petit de 450Mo nommé 100d(txt)
   - 5 placer ce fichier dans un dossier et le décompresser => 1.9Go
   - 6 placer ce fichier décompressé dans le dossier C:\\Users\\UserPC\\anaconda3\\envs\\spyder-env\\lib\\site-packages\\gensim\\test\\test_data\\
     où spyder-env est mon environnement spécial spyder (pas obligatoire)
   - 7 enfin lancer la cellule ci-dessous pour charger le fichier sur notre IDE et créer un vecteur



Le chargement d'un objet gensim prend 6 à 8 mins chaque fois, le modèle **trained** est sauvegardé et utilisé dans la suite


In [ ]:
# si c'est la première fois et que vous n'avez pas récupéré le trained.pickle, lancez les codes. sinon charger directement le modèle 
'''
# chargement d'un objet gensim de type models.keyedvectors de taille 2.500.733
start = time()

trained = load_word2vec_format('frwiki_20180420_100d.txt.bz2', binary=False)    # 6 à 8 min de chargement, patience...

end = time()
print("temps chargement modele pré-entrainé = ", end-start)
# à voir absolument le parametre unicode_errors='ignore' ou 'replace' qui sera utile par la suite pour éviter des ereurs en chargement

print(trained.vectors.shape) 
'''
'''
# sauvegarder le modèle pré-entrainé
pickle.dump(trained,open('trained.pickle','wb')) 
'''

In [30]:
# charger le modèle pré-entrainé sauvegardé
trained = pickle.load(open('trained.pickle','rb'))

In [31]:
# Test du model pré-entrainé
# tests des tokens du model pré-entrainé pour voir comme ça marche

# test de similarité du token 'papa'
print(trained.most_similar(['papa']))  # 'mamam' à 78.67% puis 'papone à 70.87% etc.....

# exemple commun de test calculatoire sur les token : +A - C => +B - ?  avec A=roi, C = homme, B=femme => ? = reine solution
print(trained.most_similar(positive=['roi', 'femme'], negative=['homme']))  # => 'reine' 1er choix à 86.49% puis 'régente' etc...

[('maman', 0.7867106199264526), ('papone', 0.7338883876800537), ('pépé', 0.7337833642959595), ('mama', 0.7087074518203735), ('baby', 0.7081818580627441), ('bébé', 0.7059006690979004), ('humduck', 0.7049368023872375), ('souriskewitz', 0.7024010419845581), ('chéri', 0.7012784481048584), ('tontaine', 0.6972444653511047)]
[('reine', 0.8649726510047913), ('régente', 0.7676728367805481), ('estrildis', 0.7609500885009766), ('nantilde', 0.7351045608520508), ('satiâh', 0.7330015897750854), ('kapes', 0.7293994426727295), ('abudok', 0.7282392978668213), ('tawananna', 0.7266699075698853), ('zakûtu', 0.7247689366340637), ('kwenthrith', 0.7246122360229492)]


## Fonction my_doc2vec() de pré-traitement

In [32]:
# fonction pour transformer un document ( ici une ligne de commentaire) en vecteur à partir des tokens qui le compose
# entrée : line_com lignes commentaires à traiter
#          model pré-entrainé
# sortie : vecteur représentant le document

def my_doc2vec(doc, trained):
    # dimension de représentation
    p = trained.vectors.shape[1]   # p = 100
    # intit du vecteur
    vec = np.zeros(p)  # array (100,)
    # nbre de tokens trouvés
    nb=0
    # traitement de chaque token de la ligne de commentaire
    for tk in doc:
        # ne traiter que les tokens reconnus
        try:
            values = trained[tk] # on récupère le vecteur du token concerné
            vec = vec + values   # on incrémente ce vecteur dans vec
            nb = nb + 1.0        # le compteur de token d'incrémente
        except:
            pass  # pour lever l'erreur si aucun token trouvé dans notre modele pré-entrainé
        # moyenne des valeurs uniquement si on a des tokens reconnus
    if (nb > 0.0):
        vec = vec/nb
    return vec  # renvoie le vecteur moyenné ou un vecteur nul si aucun token trouvé


## Pré-traitement du corpus

In [33]:
# préparer la liste corpus_sw pour modèle 
corpus_sw = data['no_stop_words'].tolist()
# corpus_sw = data['words+3'].tolist() 
# 
# traiter les documents du corpus

docsVec = list()
# pour chaque ligne du corpus nettoyé    
for doc in corpus_sw:
    # calcul de son vecteur
    vec = my_doc2vec(doc,trained)
    # ajouter dans la liste
    docsVec.append(vec)    
# transfromer en array numpy
matVec = np.array(docsVec)
print(matVec.shape)

(17199, 100)


In [34]:
# transformer en dataframe
df = pd.DataFrame(matVec, columns=["v"+str(i+1) for i in range(matVec.shape[1])])
# ajouter la classe target 'star'
df['label'] = data.star
df.head(3)

,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,v18,v19,v20,v21,v22,v23,v24,v25,...,v77,v78,v79,v80,v81,v82,v83,v84,v85,v86,v87,v88,v89,v90,v91,v92,v93,v94,v95,v96,v97,v98,v99,v100,label
0,-0.474,0.275,-0.145,-0.375,0.328,0.181,0.265,-0.038,-0.053,0.151,-0.352,0.096,-0.146,-0.327,-0.306,0.456,0.045,0.402,-0.128,0.328,-0.543,0.121,0.124,0.714,0.182,...,-0.041,-0.005,-0.294,-0.094,-0.025,-0.207,0.153,-0.118,-0.185,-0.111,-0.166,-0.068,-0.012,0.043,-0.173,0.107,0.347,-0.183,-0.252,0.014,-0.309,-0.213,0.108,-0.741,0
1,-0.469,0.281,-0.150,-0.360,0.330,0.158,0.276,-0.027,-0.048,0.131,-0.336,0.068,-0.149,-0.322,-0.299,0.443,0.060,0.394,-0.124,0.301,-0.545,0.117,0.098,0.692,0.187,...,-0.052,-0.011,-0.293,-0.102,-0.022,-0.179,0.150,-0.119,-0.165,-0.122,-0.151,-0.061,-0.009,0.042,-0.186,0.097,0.345,-0.168,-0.253,0.027,-0.298,-0.183,0.109,-0.744,0
2,-0.477,0.284,-0.156,-0.360,0.333,0.178,0.274,-0.041,-0.064,0.140,-0.368,0.091,-0.153,-0.336,-0.310,0.445,0.047,0.406,-0.130,0.317,-0.542,0.117,0.116,0.719,0.181,...,-0.046,-0.000,-0.306,-0.084,-0.028,-0.199,0.146,-0.103,-0.180,-0.110,-0.159,-0.063,-0.016,0.029,-0.184,0.109,0.331,-0.178,-0.257,0.027,-0.323,-0.211,0.106,-0.763,0


## partition en train_set et test_set

In [35]:
# partition en train_set et test_set
dfTrain, dfTest = train_test_split(df, test_size=0.2, stratify=df.label, random_state=20)
# que 2 instances au lieu de 4 car taget est dans df !

# 4- Modèle SVM 

* -Modélisation par SVM, prédiction et évaluation en test

In [36]:
# SVM avec noyau RBF par défaut
SVM = SVC(C=100, gamma=1,random_state=0)  
# ces paramètres ont été trouvés par GridSearchCV sur quelques valeurs seulement
# nous donnons le code pour cela ci-dessous mais à lancer avec parcimonie puisque les temps de calculs ont sont multipliés ! 
SVM.fit(dfTrain[dfTrain.columns[:-1]], dfTrain.label)

# sauvegarder le modèle entrainé
pickle.dump(SVM,open('SVM.pickle','wb')) 


In [37]:
# prediction en test
y_pred_SVM = SVM.predict(dfTest[dfTest.columns[:-1]])

# évaluation des performances
print(classification_report(dfTest.label, y_pred_SVM))

# Calcul et affichage de la matrice de confusion
confusion_matrix = pd.crosstab(dfTest.label, y_pred_SVM, rownames=['Classe réelle'], colnames=['Classe prédite'])
confusion_matrix

              precision    recall  f1-score   support

           0       0.76      0.81      0.78      1185
           1       0.90      0.86      0.88      2255

    accuracy                           0.84      3440
   macro avg       0.83      0.84      0.83      3440
weighted avg       0.85      0.84      0.85      3440



Classe prédite,0,1
Classe réelle,,
0,961,224
1,310,1945


In [ ]:
# recherche des best parametres avec gridsearshCV
#from sklearn.model_selection import GridSearchCV

#param_grid = {'C':[50, 100], 'kernel': ['rbf'], 'gamma': [1, 10]}  # recherche des bons hyperparams
#grid = GridSearchCV(SVC(), param_grid) # création de la grille de recherche sur le model SVC()

#grille = grid.fit(X_train, y_train)
# affichage des résultats test de GridSearchCV
#print(pd.DataFrame.from_dict(grille.cv_results_).loc[:,['params', 'mean_test_score']]) 
#print("grid.best_params :", grid.best_params_)
#svm_best= grid.best_estimator_

### Création et sauvegarde du tableau des erreurs de prédiction pour analyse ultérieure

In [38]:
# on récupère les données complètes du test_set avec indexs et star d'origine
# création d'un dataframe vide
df_analyse = pd.DataFrame()  
# recupérer tout le y_test en renommant la colonne star par star_4 pour modele 4
df_analyse['star_4'] = dfTest.label     
# ajouter une colonne d'index ordonnée de 0 à taille du df et déplacer les index d'erreurs dans une colonne du df
df_analyse.reset_index(inplace=True, drop=False)
# renommer la colonne d'index d'erreurs par index_2 pour modele 2
df_analyse.rename(columns={'index':'index_4'}, inplace=True)
# ajouter au df la colonnede prédiction réalisée sur y_test
df_analyse['pred_4'] = pd.Series(y_pred_SVM)                                        


# filtrage des erreurs de prediction par boolean indexing
 # création d'une liste boolean vérifiant sur les prédiction sont bonnes ou mauvaises       
bool_pred = [True if (df_analyse.iloc[i][1]==df_analyse.iloc[i][2]) else False for i in range(df_analyse.shape[0])]
# ajout de la liste du résultat booleen dans le tableau d'analyse des erreurs
df_analyse['bool_4'] = pd.DataFrame(bool_pred)
# création d'un dataframe des seules erreurs du modele
df_verification = df_analyse.loc[df_analyse['bool_4']==False]   # Parfait 386 erreurs aussi ici de pred facilement trouvable avec vrais index
# effacement des index non originaux copiés et sans intéret
df_verification.reset_index(inplace=True, drop=True)  


# Rajouter la colonne de commentaires originaux + colones commentaires filtrés finaux à df_ verification
list_com_vo = []
for i in range(df_verification.shape[0]):
    find_index = df_verification['index_4'][i]
    list_com_vo.append(data['Commentaire'][find_index])
    
# ajout commentaires originaux    
df_verification['comment_vo'] = pd.Series(list_com_vo)       # Parfait , les commentaires sont aux bons index !

""" pas de stop words fait sur ce modele3 donc ci-dessous inutile                                   
list_com_filtre = []
for i in range(df_verification.shape[0]):
    find_index = df_verification['index_1'][i]
    list_com_filtre.append(df['no_stop_words'][find_index])
    
# ajout commentaires traités   
df_verification['no_stop_words'] = pd.Series(list_com_filtre) 
"""


# affichage du tableau comparatif des erreurs réalisé
print("df_verification.shape = ", df_verification.shape)
df_verification.head(10)

df_verification.shape =  (534, 5)


C:\Users\UserPC\AppData\Local\Temp\ipykernel_11720\1490770386.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_verification['comment_vo'] = pd.Series(list_com_vo)       # Parfait , les commentaires sont aux bons index !


,index_4,star_4,pred_4,bool_4,comment_vo
0,10792,1,0,False,ravissantes boucles d'oreilles qui ont mis beaucoup de temps pour être livrées .
1,13463,0,1,False,lors de ma commande g mis un article dans mon panier qui etait donc disponible .... ce dernier n...
2,10437,1,0,False,plus qu'un colis qui arrivent en plusieurs livraisons . j'attends toujours le colis 2 que je n'a...
3,3331,1,0,False,déçue qu'il manque un article alors que le stock était encore valide au moment de la commande . ...
4,7577,0,1,False,j'ai reçu ma commande 4 jours après la date prévue . il manquait des produits ! je suis dans l'a...
5,5258,1,0,False,vêtement conformes à leur description bons produits je suis contente ; en revanche colis arrivé ...
6,4047,1,0,False,"commande de chaussures , j ai pas reçu la bonne paire ."
7,9152,1,0,False,j'ai commandé une taille 41 et j'ai reçu du 40 . heureusement ça me va. sinon très bien pour l'e...
8,3110,1,0,False,commande arrivée quelques jours avant la date prévue par notre sympathique facteur . arrivée en ...
9,13464,1,0,False,recu un peu tard a cause du covid 19 les postiers chez moi ils ont tous les prétexte enfin reçu ...


In [39]:
# sauvegarde des erreurs à analyser sur le model4
df_verification.to_csv('sav_verif_erreurs_model4.csv')  # 534 erreurs

### Tentative de prédiction sur un text à la demande

In [41]:
# prediction
# charger le modèle pré-entrainé sauvegardé
SVM_wiki = pickle.load(open('SVM.pickle','rb'))
# text = "la livraison est lente, le produit est abîmé"
text2 = "mauvais produit"

text_vec2 = my_doc2vec(text2, trained)  # text vectorisé de shape (100,)

# transformation du text vectorisé en shape (1, 100) avec les noms de colonnes v1 ...v 100 comme dans le modèle pré-entrainé
# récupérer la liste des noms de colonnes du dfTest entrainé
list_col = list(dfTest.columns[:-1])
# conversion du vecteur text en dataframe + transposé => shape (1, 100)
text_vector = pd.DataFrame(my_doc2vec(text2, trained)).T  # colonnes de 0 à 99 et ligne 0 du texte vectorisé !!
# renommer les noms des colonnes avec list_col
text_vector.columns = list_col

print(text_vec2)

# prédiction du text vectorisé
pred_wiki_direct_SVM = SVM_wiki.predict(text_vector)

print("La prédiction du text : ", text2, " est de ", pred_wiki_direct_SVM)

[-0.37107143  0.29697142 -0.06962143 -0.27109286  0.272       0.11660714
  0.31535714 -0.08144286 -0.05887857  0.21515714 -0.33828572  0.08315
 -0.19459286 -0.32044286 -0.20367143  0.39055     0.09951429  0.41880714
 -0.04538572  0.34085715 -0.46829286  0.07070714  0.05749286  0.70923571
  0.1292      0.12670715  0.12595    -0.22502857 -0.22791429 -0.2711
  0.27667857  0.34805     0.28662143 -0.04282857  0.57274285 -0.01487143
 -0.01197143 -0.1847     -0.16628571  0.02474286  0.14089286  0.05633572
  0.37090714 -0.08735714 -0.03256428  0.36685    -0.03162857 -0.39179286
 -0.24576429  0.0009      0.18177857 -0.12556428 -0.04155714 -0.80761428
  0.18782857  0.26592857  0.05982857 -0.00540714  0.1077     -0.20402857
  0.27002857  0.19911428 -0.09802857 -0.41096429  0.29728571 -0.28012857
  0.08625715 -0.00573571  0.05489286  0.19260714 -0.15338572 -0.05150714
 -0.38748572 -0.14592857  0.06339285  0.22604286 -0.0597      0.01913572
 -0.24955714 -0.08457857 -0.06332143 -0.21565     0.098428

In [ ]:
""" ici aussi, mauvaise prédiction => faire d'autres test, voir avec lestableaux d'erreurs, vérifier que le code est correct """

# 5- Modèle GBC avec word2vec wikipedia

In [47]:
#  GBC appliquer sur les données vectoriées avec filtrage stop word

GBC_3 = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0).fit(dfTrain[dfTrain.columns[:-1]], dfTrain.label)

# Calculer les prédictions 
y_pred_GBC_3 = GBC_3.predict(dfTest[dfTest.columns[:-1]])

# sauvegarder le modèle entrainé
pickle.dump(GBC_3,open('GBC_3.pickle','wb'))

In [48]:
# Calcul et affichage de classification_report
print('Gradient Boosting Classifier avec données vectorizées: ')
print(classification_report(dfTest.label, y_pred_GBC_3))

# Calcul et affichage de la matrice de confusion
confusion_matrix = pd.crosstab(dfTest.label, y_pred_GBC_3, rownames=['Classe réelle'], colnames=['Classe prédite'])
confusion_matrix

Gradient Boosting Classifier avec données vectorizées: 
              precision    recall  f1-score   support

           0       0.77      0.73      0.75      1185
           1       0.86      0.89      0.87      2255

    accuracy                           0.83      3440
   macro avg       0.82      0.81      0.81      3440
weighted avg       0.83      0.83      0.83      3440



Classe prédite,0,1
Classe réelle,,
0,866,319
1,258,1997


# Modèle 6 - Réseau Neurone 1
## *ANN ARTIFICEL NEURONAL NETWORK*


In [53]:
# rependre les données 
# transformer en dataframe
X = pd.DataFrame(matVec, columns=["v"+str(i+1) for i in range(matVec.shape[1])])
# ajouter la classe target 'star'
y = data.star

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0) 

## Construire le modèle 

In [55]:
""" MODELE ANN ARTIFICEL NEURONAL NETWORK --  KERAS -----------------------------------"""
inputs = Input(shape = (100), name = "Input") # couche d'entrée qui contient les dimensions de nos données X en entrée, ici 100 car 100 colonnes

dense1 = Dense(units = 50, activation = "tanh", name = "Dense_1")  # pas besoin des dimensions shape pour les couches suivantes,
dense2 = Dense(units = 20, activation = "tanh", name = "Dense_2")
dense3 = Dense(units = 8, activation = "tanh", name = "Dense_3")
dense4 = Dense(units = 2, activation = "softmax", name = "Dense_4")
x=dense1(inputs)  # -> x de type engine.keras_tensor.KerasTensor
x=dense2(x)
x=dense3(x)
outputs=dense4(x)  # tous les x suivant et le outputs sont du meme type que le 1er x
# - Les commandes suivantes permettent de finaliser la définition du modèle et d'en afficher la structure.
ANN = Model(inputs = inputs, outputs = outputs,name = "ANN")
ANN.summary()

Model: "ANN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 100)]             0         
                                                                 
 Dense_1 (Dense)             (None, 50)                5050      
                                                                 
 Dense_2 (Dense)             (None, 20)                1020      
                                                                 
 Dense_3 (Dense)             (None, 8)                 168       
                                                                 
 Dense_4 (Dense)             (None, 2)                 18        
                                                                 
Total params: 6,256
Trainable params: 6,256
Non-trainable params: 0
_________________________________________________________________


In [56]:
# compilation du modele avec "binary_crossentropy" ne marche pas !! 
ANN.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])

In [57]:
%%time
ANN.fit(X_train,y_train,epochs=500,batch_size=50,validation_split=0.1)


Epoch 1/500
217/217 [==============================] - 1s 3ms/step - loss: 0.5962 - accuracy: 0.6539 - val_loss: 0.5388 - val_accuracy: 0.6977
Epoch 2/500
217/217 [==============================] - 0s 2ms/step - loss: 0.5219 - accuracy: 0.7029 - val_loss: 0.5049 - val_accuracy: 0.7135
Epoch 3/500
217/217 [==============================] - 0s 2ms/step - loss: 0.4998 - accuracy: 0.7187 - val_loss: 0.5098 - val_accuracy: 0.7035
Epoch 4/500
217/217 [==============================] - 0s 2ms/step - loss: 0.4914 - accuracy: 0.7301 - val_loss: 0.4785 - val_accuracy: 0.7550
Epoch 5/500
217/217 [==============================] - 1s 2ms/step - loss: 0.4831 - accuracy: 0.7340 - val_loss: 0.4840 - val_accuracy: 0.7276
Epoch 6/500
217/217 [==============================] - 0s 2ms/step - loss: 0.4779 - accuracy: 0.7496 - val_loss: 0.4632 - val_accuracy: 0.7882
Epoch 7/500
217/217 [==============================] - 0s 2ms/step - loss: 0.4652 - accuracy: 0.7648 - val_loss: 0.4616 - val_accuracy: 0.7907

In [58]:
# sauvegarder le modèle entrainé
pickle.dump(ANN,open('ANN.pickle','wb'))

INFO:tensorflow:Assets written to: ram://4001e0bf-0488-4f05-9fd2-bdededca6a3f/assets


INFO:tensorflow:Assets written to: ram://4001e0bf-0488-4f05-9fd2-bdededca6a3f/assets


In [62]:
%%time 
test_pred = ANN.predict(X_test)  # => dim (5160, 2) où n =dim[0]de X_test et 2 = dim units dense_last
y_pred_class = np.argmax(test_pred,axis=1) # on prend l'index du max de chaque ligne prédite
y_test_class = y_test  # autre nom a y_test pour la suite classification_report et confusion_matrix

print(classification_report(y_test_class,y_pred_class)) 
print(confusion_matrix(y_test_class,y_pred_class))

162/162 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       0.73      0.74      0.74      1763
           1       0.86      0.86      0.86      3397

    accuracy                           0.82      5160
   macro avg       0.80      0.80      0.80      5160
weighted avg       0.82      0.82      0.82      5160

[[1301  462]
 [ 474 2923]]
Wall time: 368 ms


# Prédiction 

In [93]:
# modèle à choisir
"""
GBC_3
SVM
ANN
"""
text = "produit cassé, livraison lente, pas de réponse de service client"

# Fonction : text préparation et prédiction 

In [102]:
import spacy
# !python -m spacy download fr_core_news_md
nlp = spacy.load('fr_core_news_md')
def prediction(text, model):
    my_doc = text.lower()
    my_doc_tk = word_tokenize(my_doc)
    def lemms(corpus_tk):    
        doc = nlp(" ".join(corpus_tk))
        lemm = [token.lemma_ for token in doc]
        return lemm
    my_doc_lem = lemms(my_doc_tk)
    def stop_words_filtering(mots, stop_words) : 
        tokens = []
        for mot in mots:
            if mot not in stop_words: 
                tokens.append(mot)
        return tokens
    df_stop_word_xls = pd.read_excel('liste_no-stop-words_tokens_unique.xlsx', header=None)
    update_list_fr = list(df_stop_word_xls[0])
    # initialisation de la variable des mots vides
    stop_words = set()
    stop_words.update(update_list_fr)
    my_doc_sw = stop_words_filtering(my_doc_lem,stop_words)
    my_vec = my_doc2vec(my_doc_sw,trained)
    
#     print(my_doc_sw)
    pred_my_doc = model.predict(my_vec.reshape(1,-1))
    return pred_my_doc

In [103]:
print(prediction(text,SVM))
print(prediction(text,GBC_3))
print(prediction(text,ANN))
# 1 - positive
# 0 - negative 
# ANN nous donne un résultat proba 

C:\LQ\softwares\Anaconda\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  "X does not have valid feature names, but"


[1]


C:\LQ\softwares\Anaconda\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  "X does not have valid feature names, but"


[1]
1/1 [==============================] - 0s 19ms/step
[[0.9690458  0.03095415]]


# Modèle 7 - Réseau Neurone

In [70]:
# chargement du bon csv des données en français 
df = pd.read_csv('review_trust_fr_lemmantiser_word+2_VF.csv', sep=',') 
# utiliser une vectorization proposée par keras
df.drop(df.iloc[:,:1], axis=1, inplace=True)
df.head(2)

,Commentaire,star,source,company,index_org,star_org,date,tokens,lemmatiser,words+2,no_stop_words
0,"bonjour , ca doit faire 5 ans environ que je suis membre showrooprive . je n ’ ai jamais eu de s...",0,TrustPilot,ShowRoom,0,1,2021-06-20 00:00:00+00:00,"['bonjour', 'ca', 'doit', 'faire', 'ans', 'environ', 'que', 'je', 'suis', 'membre', 'showroopriv...","['bonjour', 'ca', 'devoir', 'faire', 'ans', 'environ', 'que', 'je', 'être', 'membre', 'showroopr...","['bonjour', 'devoir', 'faire', 'ans', 'environ', 'que', 'être', 'membre', 'showroopriv', 'avoir'...","['bonjour', 'devoir', 'faire', 'ans', 'environ', 'que', 'être', 'membre', 'showroopriv', 'avoir'..."
1,vente lacoste article manquant photo prise sur 6 articles la moitié livrée sans explication manq...,0,TrustPilot,ShowRoom,1,1,2021-06-20 00:00:00+00:00,"['vente', 'lacoste', 'article', 'manquant', 'photo', 'prise', 'sur', 'articles', 'la', 'moitié',...","['vente', 'lacost', 'article', 'manquant', 'photo', 'prendre', 'sur', 'article', 'le', 'moitié',...","['vente', 'lacost', 'article', 'manquant', 'photo', 'prendre', 'sur', 'article', 'moitié', 'livr...","['vente', 'lacost', 'article', 'manquant', 'photo', 'prendre', 'sur', 'article', 'moitié', 'livr..."


In [65]:
# # enlever les informations inutiles
# df = df.drop(['source', 'company','date','tokens','words+2','lemmatiser'], axis = 1)


In [66]:
# df.head(2)

,Commentaire,star,index_org,star_org,no_stop_words
0,"bonjour , ca doit faire 5 ans environ que je suis membre showrooprive . je n ’ ai jamais eu de s...",0,0,1,"['bonjour', 'devoir', 'faire', 'ans', 'environ', 'que', 'être', 'membre', 'showroopriv', 'avoir'..."
1,vente lacoste article manquant photo prise sur 6 articles la moitié livrée sans explication manq...,0,1,1,"['vente', 'lacost', 'article', 'manquant', 'photo', 'prendre', 'sur', 'article', 'moitié', 'livr..."


In [71]:
df['comment'] = df['no_stop_words'].apply(lambda x : x.replace(',',' ').replace("'","").replace("  "," ").replace("[","").replace("]",""))
df['comment'][0]

'bonjour devoir faire ans environ que être membre showroopriv avoir jamais avoir souci particulier part petit bricole par contre depuis dernier mois vrai catastrophe avoir avoir commande annuler affilier demander vente être bien réel histoire nous faire payer que showroom joue avec notre argent faire intérêt bancaire sur notre dos ère commande cafés annuler mois après commande don jennifer annuler mois après commande techwood annuler mois après apprendre toujours par mail même message bateau malgré tout effort bla bla bla bien évidemment pas dédommagement ben non hein pourquoi faire doute que site être aussi fiable sérieux que début final aller juste rester fidèle sur autre site marchand très décevant plus tout confiance recommande plus tout site faire part mon entourage réseau social afin éviter que autre personne connaître même type expérience'

In [72]:
# liste de paramètres
# DATASET
TRAIN_SIZE = 0.8
# WORD2VEC 
W2V_SIZE = 300
W2V_WINDOW = 7
W2V_EPOCH = 32
W2V_MIN_COUNT = 10

# KERAS
SEQUENCE_LENGTH = 300
EPOCHS = 8
BATCH_SIZE = 1024


In [73]:
df = df[['comment','star']]
df =df.rename({'comment':'text', 'star':'target'},axis = 1)
df.head()

,text,target
0,bonjour devoir faire ans environ que être membre showroopriv avoir jamais avoir souci particulie...,0
1,vente lacost article manquant photo prendre sur article moitié livrer sans explication manqu res...,0
2,vente lacost honteux article erroné article manquant pas bon livraison retard service client dés...,0
3,avoir commander mule marque très déçu produit étiquette coller sur mule donc tâche produire très...,0
4,commande téléphone etat avoir livraison vieux téléphone pourrir sans batterie rayer partout supe...,0


In [74]:
# split train and test
df_train, df_test = train_test_split(df, test_size=1-TRAIN_SIZE, random_state = 42)
print('train size: ', len(df_train))
print('test size: ', len(df_test))

train size:  13759
test size:  3440


## Tokenize text

In [75]:
# package utilisé
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
# différente version keras le nom de keras_precessing.sequence peut être différent comme keras.precessing.sequence
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Embedding, Flatten, Conv1D, MaxPooling1D, LSTM
from keras import utils
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [76]:
%%time
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_train.text)
vocab_size = len(tokenizer.word_index)+1
print("total words ", vocab_size)

total words  6798
Wall time: 495 ms


In [77]:
%%time
x_train = pad_sequences(tokenizer.texts_to_sequences(df_train.text), maxlen=SEQUENCE_LENGTH)
x_test = pad_sequences(tokenizer.texts_to_sequences(df_test.text), maxlen=SEQUENCE_LENGTH)
# pad_sequences: Pads sequences to the same length

Wall time: 511 ms


In [78]:
y_train = df_train.target.values.reshape(-1,1)
y_test = df_test.target.values.reshape(-1,1)
print('y_train shape ',y_train.shape)
print('y_test shape ', y_test.shape)

y_train shape  (13759, 1)
y_test shape  (3440, 1)


In [79]:
# vérification des dimentions
print("x_train", x_train.shape)
print("y_train", y_train.shape)
print()
print("x_test", x_test.shape)
print("y_test", y_test.shape)

x_train (13759, 300)
y_train (13759, 1)

x_test (3440, 300)
y_test (3440, 1)


## Embedding layer

### importer le résultat - word2vec 

In [80]:
from gensim.models.keyedvectors import load_word2vec_format
from gensim.test.utils import datapath
w2v_model =  pickle.load(open('trained.pickle','rb'))

In [82]:
embedding_matrix = np.zeros((vocab_size, W2V_SIZE))
for word, i in tokenizer.word_index.items():
    if word in w2v_model.key_to_index:
        embedding_matrix[i] = w2v_model.key_to_index[word]
print(embedding_matrix.shape)
        

(6798, 300)


In [83]:
embedding_layer = Embedding(vocab_size, W2V_SIZE, weights=[embedding_matrix], input_length=SEQUENCE_LENGTH, trainable=False)

## build model

In [84]:
model = Sequential()
model.add(embedding_layer)
model.add(Dropout(0.5))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 300)          2039400   
                                                                 
 dropout (Dropout)           (None, 300, 300)          0         
                                                                 
 lstm (LSTM)                 (None, 100)               160400    
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 2,199,901
Trainable params: 160,501
Non-trainable params: 2,039,400
_________________________________________________________________


## compile model

In [85]:
model.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

## Callbacks

In [86]:
callbacks = [ ReduceLROnPlateau(monitor='val_loss', patience=5, cooldown=0),
              EarlyStopping(monitor='val_accuracy', min_delta=1e-4, patience=5)]

# Train

In [87]:
%%time
history = model.fit(x_train, y_train,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    
                    validation_split=0.1,
                    verbose=1,
                    callbacks=callbacks)

Epoch 1/8
13/13 [==============================] - 165s 12s/step - loss: 0.7017 - accuracy: 0.5943 - val_loss: 0.6331 - val_accuracy: 0.6679 - lr: 0.0010
Epoch 2/8
13/13 [==============================] - 204s 16s/step - loss: 0.6255 - accuracy: 0.6559 - val_loss: 1.0784 - val_accuracy: 0.6679 - lr: 0.0010
Epoch 3/8
13/13 [==============================] - 244s 19s/step - loss: 0.5031 - accuracy: 0.7229 - val_loss: 1.0060 - val_accuracy: 0.6679 - lr: 0.0010
Epoch 4/8
13/13 [==============================] - 303s 23s/step - loss: 0.4222 - accuracy: 0.8262 - val_loss: 0.7388 - val_accuracy: 0.6701 - lr: 0.0010
Epoch 5/8
13/13 [==============================] - 344s 26s/step - loss: 0.3959 - accuracy: 0.8423 - val_loss: 0.5990 - val_accuracy: 0.6701 - lr: 0.0010
Epoch 6/8
13/13 [==============================] - 380s 29s/step - loss: 0.3848 - accuracy: 0.8404 - val_loss: 0.5228 - val_accuracy: 0.6701 - lr: 0.0010
Epoch 7/8
13/13 [==============================] - 390s 30s/step - loss: 0.3

In [88]:
%%time
score = model.evaluate(x_test, y_test, batch_size=BATCH_SIZE)
print()
print("ACCURACY:",score[1])
print("LOSS:",score[0])
y_pred= model.predict(x_test)

4/4 [==============================] - 5s 1s/step - loss: 0.4827 - accuracy: 0.6779

ACCURACY: 0.6779069900512695
LOSS: 0.4827301800251007
108/108 [==============================] - 42s 387ms/step
Wall time: 47.7 s
